# Geocoding


### Table of Content

1. [Import Libraries and the dataset](#import)
2. 

#### 1. Import Libraries and the dataset. <a id = 'import'></a>

In [2]:
#Libraies imported
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from bokeh.plotting import figure, show, output_notebook, output_file, reset_output
from bokeh.layouts import gridplot
from bokeh.models import HoverTool
import osmnx as ox
import geopandas as gpd
import re
from geopandas.tools import geocode
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [4]:
#Dataset imported
real_estate = pd.read_csv('../Datasets/')
#Reset Index
real_estate.reset_index(drop=True)

/var/folders/kv/4xjj0g696_lcyc7ms5pdd2t00000gn/T/ipykernel_14570/2421691908.py:2: DtypeWarning: Columns (8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  real_estate = pd.read_csv('../Datasets/Real_Estate_Sales_2001-2020_GL.csv')


,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location
0,2020348,2020,09/13/2021,Ansonia,230 WAKELEE AVE,150500.0,325000.0,0.463000,Commercial,NaN,NaN,NaN,NaN,NaN
1,20002,2020,10/02/2020,Ashford,390 TURNPIKE RD,253000.0,430000.0,0.588300,Residential,Single Family,NaN,NaN,NaN,NaN
2,200212,2020,03/09/2021,Avon,5 CHESTNUT DRIVE,130400.0,179900.0,0.724800,Residential,Condo,NaN,NaN,NaN,NaN
3,200243,2020,04/13/2021,Avon,111 NORTHINGTON DRIVE,619290.0,890000.0,0.695800,Residential,Single Family,NaN,NaN,NaN,NaN
4,200377,2020,07/02/2021,Avon,70 FAR HILLS DRIVE,862330.0,1447500.0,0.595700,Residential,Single Family,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997208,190272,2019,06/24/2020,New London,4 BISHOP CT,60410.0,53100.0,1.137665,Single Family,Single Family,14 - Foreclosure,NaN,NaN,NaN
997209,190284,2019,11/27/2019,Waterbury,126 PERKINS AVE,68280.0,76000.0,0.898400,Single Family,Single Family,25 - Other,PRIVATE SALE,NaN,NaN
997210,190129,2019,04/27/2020,Windsor Locks,19 HATHAWAY ST,121450.0,210000.0,0.578300,Single Family,Single Family,NaN,NaN,NaN,NaN
997211,190504,2019,06/03/2020,Middletown,8 BYSTREK DR,203360.0,280000.0,0.726300,Single Family,Single Family,NaN,NaN,NaN,NaN


The dataset is imported. 

Because the dataset is continually being updated after retrieving the coordinates from Open Street Map API, the  latest version of the dataset should be imported, not the raw dataset. 

The latest updated dataset can be downloaded here:

Checking the format of the dataframe. 

In [23]:
import pandas as pd
import re
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

# Initialize the geocoder
geolocator = Nominatim(user_agent="bootcamp student")

real_estate = pd.DataFrame(real_estate)

max_retries = 1
last_successful_index = None  # Initialize to None

for index, row in real_estate.iterrows():
    if pd.notnull(row["latitude"]) or pd.notnull(row["longitude"]):
        # Skip to the next row if either Latitud and Longitude are already present
        continue
    address = row["Full Address"]
    retries = 0
    while retries < max_retries:
        try:
            location = geolocator.geocode(address, timeout=10)
            if location is not None:
                if location.latitude and location.longitude:
                    real_estate.at[index, "latitude"] = location.latitude
                    real_estate.at[index, "longitude"] = location.longitude
                    last_successful_index = index  # Update the index of last successful geocode
                break  # Successful geocode, break out of the loop
            else:
                break  # No more retries needed if geocoding failed
        except GeocoderTimedOut:
            retries += 1
            print("Geocoding timed out for:", address)
            print("Retrying... (Attempt", retries, "of", max_retries, ")")
    else:
        real_estate.at[index, "latitude"] = None
        real_estate.at[index, "longitude"] = None
        print("Max retries reached for:", address)

# Print the index of the last successful geocode
print("Last successful geocode index:", last_successful_index)


KeyboardInterrupt: 